In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Flatten
import pickle

2022-05-29 20:55:41.087580: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-29 20:55:41.087600: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
IoT = pd.read_csv('../../data/test_heterogeneous.csv')
IoT = IoT.iloc[0:5000]

IoT = IoT.dropna()

encoder=LabelEncoder()
IoT['fridge_temp_condition']=encoder.fit_transform(IoT['fridge_temp_condition'])
IoT['door_state']=encoder.fit_transform(IoT['door_state'])
IoT['door_sphone_signal']=encoder.fit_transform(IoT['door_sphone_signal'])
IoT['door_type']=encoder.fit_transform(IoT['door_type'])
IoT['ml_light_status']=encoder.fit_transform(IoT['ml_light_status'])

IoT.head()

,date-time,fridge_temperature,fridge_temp_condition,door_state,door_sphone_signal,door_type,gps_latitude,gps_longitude,modbus_FC1_Read_Input_Register,modbus_FC2_Read_Discrete_Value,modbus_FC3_Read_Holding_Register,modbus_FC4_Read_Coil,ml_motion_status,ml_light_status,thermostat_current_temperature,thermostat_status,weather_temperature,weather_pressure,weather_humidity
0,31-03-19~12:36:52,13.10,0,0,0,2,0.0,10.0,11287,3537,27844,30527,1,1,25.0,0,31.788508,1.035,32.036579
1,31-03-19~12:36:53,8.65,0,0,0,2,0.0,10.0,11287,3537,27844,30527,0,0,25.0,0,41.630997,1.035,30.886165
2,31-03-19~12:36:54,2.00,1,0,0,2,0.0,10.0,11287,3537,27844,30527,0,0,25.0,0,42.256959,1.035,19.755908
3,31-03-19~12:36:55,4.80,1,0,0,2,0.0,10.0,53287,1463,33518,23014,1,1,25.0,0,49.116581,1.035,78.949621
4,31-03-19~12:36:56,10.70,0,0,0,2,0.0,10.0,53287,1463,33518,23014,1,1,25.0,0,24.017085,1.035,40.001059


In [3]:
x = IoT.drop(['door_type', 'date-time'], axis=1)
y = IoT['door_type']

features=len(x.columns)

x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2)

In [4]:
model1 = Sequential()
    
model1.add(LSTM(units = 128, activation='tanh', return_sequences=True, input_shape = (features,1)))
model1.add(Dropout(0.2))
model1.add(LSTM(units = 64, activation='tanh', return_sequences=True))
model1.add(Dropout(0.2))
model1.add(Dense(1, activation='sigmoid')) 
model1.add(Flatten())
    
model1.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

2022-05-29 20:55:44.016630: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-29 20:55:44.016811: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-29 20:55:44.016858: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-05-29 20:55:44.016901: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-05-29 20:55:44.016942: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [5]:
model1.save('../h5s/LSTM.h5') 
# filename = '../h5s/LSTM.h5'
# pickle.dump(model1, open(filename, 'wb'))

In [6]:
model1.fit(x_train, np.asarray(y_train).astype('float32').reshape((-1,1)), epochs = 5, batch_size=32, validation_data=(x_test, np.asarray(y_test).astype('float32').reshape((-1,1))))

Epoch 1/5
125/125 [==============================] - 4s 17ms/step - loss: -2.8661 - accuracy: 0.0000e+00 - val_loss: -4.9286 - val_accuracy: 0.0000e+00
Epoch 2/5
125/125 [==============================] - 2s 13ms/step - loss: -4.8113 - accuracy: 0.0000e+00 - val_loss: -6.5951 - val_accuracy: 0.0000e+00
Epoch 3/5
125/125 [==============================] - 2s 13ms/step - loss: -6.1407 - accuracy: 0.0060 - val_loss: -6.8200 - val_accuracy: 0.0000e+00
Epoch 4/5
125/125 [==============================] - 2s 13ms/step - loss: -6.3035 - accuracy: 0.2333 - val_loss: -6.8863 - val_accuracy: 0.4640
Epoch 5/5
125/125 [==============================] - 2s 13ms/step - loss: -6.4196 - accuracy: 0.5477 - val_loss: -6.8685 - val_accuracy: 0.6930


In [7]:
predictions = model1.predict(x_test)
y_pred = [round(x[0]) for x in predictions]
y_pred

32/32 [==============================] - 1s 6ms/step


[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [8]:
for i in range(3):
	print(x_test.values[i].tolist(), '=>', y_pred[i], '( expected ', y_train.values[i], ')')

[7.95, 0.0, 0.0, 0.0, 12.51037918, 23.50418406, 53272.0, 48811.0, 45546.0, 4585.0, 1.0, 1.0, 27.17484865, 1.0, 39.19767305, 2.227158853, 57.30346219] => 1 ( expected  3 )
[2.2, 1.0, 0.0, 0.0, 319.7678523, 326.2472473, 5761.0, 63389.0, 5925.0, 2573.0, 0.0, 0.0, 27.8047971, 1.0, 29.86562387, 7.240251514, 20.90448955] => 1 ( expected  2 )
[8.1, 0.0, 0.0, 0.0, 448.3890858, 452.9896251, 358.0, 26386.0, 43229.0, 46854.0, 0.0, 0.0, 27.82001516, 1.0, 38.50496047, 9.545555583, 41.92646972] => 1 ( expected  0 )


In [9]:
score, acc = model1.evaluate(x_test, y_test, batch_size=1)

print('Test score:', score)
print('Test accuracy:', acc)

1000/1000 [==============================] - 3s 2ms/step - loss: -6.8685 - accuracy: 0.6750
Test score: -6.868523597717285
Test accuracy: 0.675000011920929


In [10]:
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))
recall = recall_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))
f1Score = f1_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))
print('Test accuracy:', accuracy)
print('Test precision:', precision)
print('Test recall:', recall)
print('Test f1Score:', f1Score)

Test accuracy: 0.097
Test precision: 0.097
Test recall: 1.0
Test f1Score: 0.17684594348222427
